In [1]:
import sframe
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal  # multivariate Gaussian distribution
import copy                                  # Deep copies
# image handling library
from PIL import Image
from io import BytesIO
%matplotlib inline

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1514910618.log


### Implementing the EM algorithm for Gaussian mixture models

In this section, you will implement the EM algorithm. We will take the following steps:

* Create some synthetic data.
* Provide a log likelihood function for this model.
* Implement the EM algorithm.
* Visualize the progress of the parameters during the course of running EM.
* Visualize the convergence of the model.

**Log likelihood**. We provide a function to calculate log likelihood for mixture of Gaussians. The log likelihood quantifies the probability of observing a given set of data under a particular setting of the parameters in our model. We will use this to assess convergence of our EM algorithm; specifically, we will keep looping through EM update steps until the log likelihood ceases to increase at a certain rate.


In [2]:
def log_sum_exp(Z):
    """compute log(\sum_i exp(Z_i)) for some array Z."""
    return np.max(Z) + np.log(np.sum(np.exp(Z - np.max(Z))))

def loglikelihood(data, weights, means, covs):
    """Compute the loglikelihood of the data for a Gaussian mixture model with the given parameters."""
    num_clusters = len(means)
    num_dim = len(data[0])
    
    ll = 0
    for d in data:
        Z = np.zeros(num_clusters)
        for k in range(num_clusters):
            # Compute (x - mu)^T*Sigma^{-1}*(x-mu)
            delta = np.array(d) - means[k]
            exponent_term = np.dot(delta.T, np.dot(np.linalg.inv(covs[k]), delta))
            
            # Compute loglikelihood contribution for this data point and this cluster
            Z[k] += np.log(weights[k])
            Z[k] -= 1/2. * (num_dim * np.log(2*np.pi) + np.log(np.linalg.det(covs[k])) + exponent_term)
            
        # Increment loglikelihood contribution of this data point across all clusters
        ll += log_sum_exp(Z)
    return ll